In [66]:
import torchvision
import pretrainedmodels
import torch
import pretrainedmodels.utils as utils
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image

In [2]:
model = pretrainedmodels.__dict__['densenet121'](num_classes=1000, pretrained='imagenet')

load_img = utils.LoadImage()

# transformations depending on the model
# rescale, center crop, normalize, and others (ex: ToBGR, ToRange255)
tf_img = utils.TransformImage(model) 

path_img = '/home/dattran/data/xray/00000001_000.png'

input_img = load_img(path_img)
input_tensor = tf_img(input_img)         # 3x400x225 -> 3x299x299 size may differ
input_tensor = input_tensor.unsqueeze(0) # 3x299x299 -> 1x3x299x299
input = torch.autograd.Variable(input_tensor,
    requires_grad=False)

In [3]:
output_logits = model(input)
output_logits

Variable containing:
-7.6336e-01 -2.6342e-01 -4.4826e-01  ...  -3.6672e-01  1.1599e+00  1.1597e+00
[torch.FloatTensor of size 1x1000]

In [5]:
feature = model.features(input)

In [26]:
pretrainedmodels.pretrained_settings['densenet121']
# pretrainedmodels.pretrained_settings['senet154']

{'imagenet': {'input_range': [0, 1],
  'input_size': [3, 224, 224],
  'input_space': 'RGB',
  'mean': [0.485, 0.456, 0.406],
  'num_classes': 1000,
  'std': [0.229, 0.224, 0.225],
  'url': 'http://data.lip6.fr/cadene/pretrainedmodels/senet154-c7b49a05.pth'}}

In [25]:
f=pretrainedmodels.__dict__['densenet121']

Signature: f(num_classes=1000, pretrained='imagenet')
Docstring:
Densenet-121 model from
`"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`
File:      ~/miniconda2/envs/dat/lib/python3.6/site-packages/pretrainedmodels/models/torchvision_models.py
Type:      function


In [76]:
model = pretrainedmodels.__dict__['densenet121'](num_classes=1000, pretrained='imagenet')
model.eval()
normalize = transforms.Normalize(model.mean, model.std)
transform = transforms.Compose([
        transforms.Resize(264),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(size=512),
        transforms.ColorJitter(0.15, 0.15),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        normalize
    ])
image_file = '/home/dattran/data/xray/00000001_000.png'
image = Image.open(image_file).convert('RGB')
input_tensor = transform(image).unsqueeze(0)
input = torch.autograd.Variable(input_tensor,
    requires_grad=False)
# output = model(input)
# output

In [77]:
class Densenet121(nn.Module):
    
    def __init__(self, out_size):
        super(Densenet121, self).__init__()
        model = models.densenet121(pretrained=True)  
        self.features = nn.Sequential(*list(model.features.children()))
        self.classifier = model.classifier
#         num_ftrs = self.densenet121.classifier.in_features
#         self.classifier = nn.Sequential(
#             nn.Linear(num_ftrs, out_size),
#             nn.Sigmoid()
#         )
        
    
    def forward(self, x):
        x = self.features(x) # 1x1024x7x7
        print('Feature', x.size())
        x = F.relu(x, inplace=True) # 1x1024x7x7
        print('Relu', x.size())
        x = F.avg_pool2d(x, kernel_size=16, stride=1) # 1x1024x1x1
        x = x.view(x.size(0), -1) # 1x1024
#         x = self.classifier(x)
        print('avg pool', x.size())
        x = self.classifier(x) # 1x1000
        print('classify', x.size())
        return x
    
model = Densenet121(14)
output = model(input)
# output.size()

Feature torch.Size([1, 1024, 16, 16])
Relu torch.Size([1, 1024, 16, 16])
avg pool torch.Size([1, 1024])
classify torch.Size([1, 1000])


In [59]:
# model = models.densenet121()
# for m in model.modules():
#     print(m)
#     print('====')

Docstring:
avg_pool2d(input, kernel_size, stride=None, padding=0, ceil_mode=False, count_include_pad=True) -> Variable

Applies 2D average-pooling operation in kh x kw regions by step size
dh x dw steps. The number of output features is equal to the number of
input planes.

See :class:`~torch.nn.AvgPool2d` for details and output shape.

Args:
    input: input tensor (minibatch x in_channels x iH x iW)
    kernel_size: size of the pooling region. Can be a single number or a
      tuple (kH x kW)
    stride: stride of the pooling operation. Can be a single number or a
      tuple (sH, sW). Default is equal to kernel size
    padding: implicit zero paddings on both sides of the input. Can be a
      single number or a tuple (padH, padW). Default: 0
    ceil_mode: when True, will use `ceil` instead of `floor` in the formula
        to compute the output shape. Default: ``False``
    count_include_pad: when True, will include the zero-padding in th
        averaging calculation. Default: ``